In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/svhndataset/train_32x32.mat
/kaggle/input/svhndataset/test_32x32.mat
/kaggle/input/svhndataset/extra_32x32.mat


In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from scipy.io import loadmat
import os



In [21]:
# Load the dataset from the input directory
train_data = loadmat('/kaggle/input/svhndataset/train_32x32.mat')
test_data = loadmat('/kaggle/input/svhndataset/test_32x32.mat')



In [22]:
# Extract the features and labels from the dataset
X_train = train_data['X']
y_train = train_data['y']
X_test = test_data['X']
y_test = test_data['y']


In [23]:
# Reshape the data to match the input format (num_samples, height, width, channels)
X_train = np.transpose(X_train, (3, 0, 1, 2))
X_test = np.transpose(X_test, (3, 0, 1, 2))



In [24]:
# Replace label '10' with '0' for digit '0'
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

In [25]:
# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [26]:
# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [27]:
# Create an image data generator with data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True
)
datagen.fit(X_train)

In [ ]:
# Step 5: CNN Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])
# Compile the model with Adam optimizer
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 6: Train the Model
# Train the model using the augmented data generator
history = model.fit(datagen.flow(X_train, y_train, batch_size=64),
                    validation_data=(X_test, y_test),
                    steps_per_epoch=len(X_train) // 64,
                    epochs=15)

In [ ]:
# Step 7: Evaluate the Model
# Evaluate the model on the test dataset
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

# Get model predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Classification Report
print("Classification Report:\n", classification_report(y_true, y_pred_classes))

# Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred_classes))

In [ ]:
# Step 8: Save the Model to Kaggle Working Directory
# Save the trained model
model.save('/kaggle/working/svhn_cnn_model.h5')

In [ ]:
# Step 9: Visualize Training Results
# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 4))

# Plot Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Save the plots to Kaggle working directory
plt.savefig('/kaggle/working/training_results.png')

# Display the plots
plt.show()

# Confusion Matrix Visualization
plt.figure(figsize=(10, 7))
plt.imshow(confusion_matrix(y_true, y_pred_classes), cmap='Blues')
plt.title("Confusion Matrix")
plt.colorbar()
plt.savefig('/kaggle/working/confusion_matrix.png')
plt.show()